In [65]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import * 
from scipy.stats import wilcoxon

In [14]:
def create_rotation_df(subj, rois, sess):
    cols = ['source', 'base', 'target', 'U', 'distance']
    rotations = np.zeros(((len(rois) * (len(rois))) * 2, len(cols)), dtype=object)
    i = 0 
    for roi_source in rois.keys():
        mds_source = np.load(f'/media/harveylab/STORAGE1 nA/stan-thesis/projects/NSD/rsa/mds/{subj}/{subj}_{sess}_{roi_source}_mds.npy', allow_pickle=False)
        for roi_target in rois.keys():
            
            for j in range(2):
                mds_target = np.load(f'/media/harveylab/STORAGE1 nA/stan-thesis/projects/NSD/rsa/mds/{subj}/{subj}_{sess}_{roi_target}_mds.npy', allow_pickle=False)
                    # flip the target when j is at 1, does nothing when j = 0 
                rotations[i, 1] = roi_target
                if j == 1:
                    mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                    roi_target = roi_target + "_flipped"
                U, t= kabsch2D(mds_source, mds_target, translate=True )
                rotations[i, 0] = roi_source
                rotations[i, 2] = roi_target
                rotations[i, 3] = U
                rotated_source = rotate(mds_source, U)
                rotations[i, 4] = avg_distance(rotated_source, mds_target, t)
                i += 1
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'distance': 'float32'}) # needed for later indexing
    return df

def filter_df(df):
 # get the index of the lowest error for each pair in each ROI
    indx_to_keep = df.groupby(['source', 'base'])['distance'].idxmin()
    df_filtered = df.loc[indx_to_keep]
    return df_filtered




In [23]:
df_01 = filter_df(create_rotation_df('subj01', rois, sessions[0]))
df_01

,source,base,target,U,distance
208,LO-1,LO-1,LO-1,"[[1.0, 0.0], [0.0, 1.0]]",0.000000
210,LO-1,LO-2,LO-2,"[[-0.1208663, 0.99266875], [-0.99266875, -0.12...",0.508327
205,LO-1,PHC-1,PHC-1_flipped,"[[-0.9988094208821031, -0.04878258661815399], ...",0.696014
206,LO-1,PHC-2,PHC-2,"[[0.99697673, 0.077701], [-0.077701, 0.99697673]]",0.422869
213,LO-1,TO-1,TO-1_flipped,"[[-0.5939543841104284, -0.8044987194495719], [...",0.596031
...,...,...,...,...,...
74,hV4,V2,V2,"[[0.8837158, -0.46802384], [0.46802384, 0.8837...",0.403996
77,hV4,V3,V3_flipped,"[[0.9636596321562075, 0.2671331378780315], [-0...",0.683796
80,hV4,VO-1,VO-1,"[[0.3004219, 0.95380646], [-0.95380646, 0.3004...",0.658946
82,hV4,VO-2,VO-2,"[[0.79449725, -0.6072677], [0.6072677, 0.79449...",0.361634


In [75]:
def get_distances_df(df, rois):
    cols = list(rois.keys())
    distances = np.zeros((len(cols), len(cols)), dtype=object)
    distances_df = pd.DataFrame(distances, columns=cols, index=cols)
    for roi_x in cols:
        for roi_y in cols: 
            distances_df.loc[roi_x, roi_y] = df.loc[(df["source"] == roi_x) & (df["base"] == roi_y)]['distance'].values[0]

    return distances_df

In [77]:
distances_df = get_distances_df(df_01, rois)

In [92]:
distances_df

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,0.0,0.598965,0.541289,0.670393,0.540082,0.649514,0.638079,0.721072,0.722144,0.814744,0.844043,0.78566
V2,0.598965,0.0,0.671916,0.403996,0.657127,0.43159,0.70088,0.767078,0.767561,0.815147,0.844827,0.681595
V3,0.541289,0.671916,0.0,0.683796,0.477352,0.657391,0.645593,0.727906,0.729323,0.808509,0.841907,0.782702
hV4,0.670393,0.403996,0.683796,0.0,0.658946,0.361634,0.684271,0.757003,0.775163,0.831347,0.848057,0.62223
VO-1,0.540082,0.657127,0.477352,0.658946,0.0,0.611037,0.58079,0.724644,0.742679,0.816197,0.850097,0.740976
VO-2,0.649514,0.43159,0.657391,0.361634,0.611037,0.0,0.689122,0.740976,0.763028,0.83048,0.846797,0.62731
PHC-1,0.638079,0.70088,0.645593,0.684271,0.58079,0.689122,0.0,0.615895,0.696014,0.75906,0.782254,0.68405
PHC-2,0.721072,0.767078,0.727906,0.757003,0.724644,0.740976,0.615895,0.0,0.422869,0.570699,0.577971,0.67742
LO-1,0.722144,0.767561,0.729323,0.775163,0.742679,0.763028,0.696014,0.422869,0.0,0.508327,0.596031,0.74947
LO-2,0.814744,0.815147,0.808509,0.831347,0.816197,0.83048,0.75906,0.570699,0.508327,0.0,0.44698,0.712025


In [90]:
def compute_wilcoxon(df, rois):
    cols = rois.keys()
    T_values = np.zeros((len(cols), len(cols)), dtype=object)
    p_values = np.ones((len(cols), len(cols)),  dtype=object)
    T_values_df = pd.DataFrame(T_values, columns =cols, index=cols)
    p_values_df = pd.DataFrame(p_values, columns= cols, index=cols)
    for roi_x in cols:
        for roi_y in cols: 
            if roi_x == roi_y:
                continue
            x = df[roi_x][(df.index != roi_x) & (df.index != roi_y)]
            y = df[roi_y][(df.index != roi_x) & (df.index != roi_y)]
            res = wilcoxon(x, y)
            T_values_df.loc[roi_x, roi_y] = res.statistic
            p_values_df.loc[roi_x, roi_y] = res.pvalue

    return T_values_df, p_values_df
    

In [91]:
T_values, p_values = compute_wilcoxon(distances_df, rois)
p_values

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,1,0.845703,0.232422,1.0,0.492188,1.0,0.845703,0.695312,1.0,0.492188,0.275391,0.769531
V2,0.845703,1,0.921875,1.0,1.0,0.232422,0.695312,1.0,0.695312,0.492188,0.322266,0.431641
V3,0.232422,0.921875,1,0.921875,0.130859,1.0,0.625,0.695312,1.0,0.695312,0.275391,0.921875
hV4,1.0,1.0,0.921875,1,0.921875,0.275391,0.625,0.921875,0.556641,0.556641,0.431641,0.556641
VO-1,0.492188,1.0,0.130859,0.921875,1,1.0,1.0,1.0,1.0,0.492188,0.275391,0.769531
VO-2,1.0,0.232422,1.0,0.275391,1.0,1,0.695312,0.921875,0.556641,0.556641,0.232422,0.232422
PHC-1,0.845703,0.695312,0.625,0.625,1.0,0.695312,1,1.0,1.0,0.492188,0.193359,0.431641
PHC-2,0.695312,1.0,0.695312,0.921875,1.0,0.921875,1.0,1,0.048828,0.064453,0.037109,0.375
LO-1,1.0,0.695312,1.0,0.556641,1.0,0.556641,1.0,0.048828,1,0.105469,0.009766,0.695312
LO-2,0.492188,0.492188,0.695312,0.556641,0.492188,0.556641,0.492188,0.064453,0.105469,1,0.001953,0.769531


In [93]:
T_values

,V1,V2,V3,hV4,VO-1,VO-2,PHC-1,PHC-2,LO-1,LO-2,TO-1,TO-2
V1,0,25.0,15.0,27.0,20.0,27.0,25.0,23.0,27.0,20.0,16.0,24.0
V2,25.0,0,26.0,27.0,27.0,15.0,23.0,27.0,23.0,20.0,17.0,19.0
V3,15.0,26.0,0,26.0,12.0,27.0,22.0,23.0,27.0,23.0,16.0,26.0
hV4,27.0,27.0,26.0,0,26.0,16.0,22.0,26.0,21.0,21.0,19.0,21.0
VO-1,20.0,27.0,12.0,26.0,0,27.0,27.0,27.0,27.0,20.0,16.0,24.0
VO-2,27.0,15.0,27.0,16.0,27.0,0,23.0,26.0,21.0,21.0,15.0,15.0
PHC-1,25.0,23.0,22.0,22.0,27.0,23.0,0,27.0,27.0,20.0,14.0,19.0
PHC-2,23.0,27.0,23.0,26.0,27.0,26.0,27.0,0,8.0,9.0,7.0,18.0
LO-1,27.0,23.0,27.0,21.0,27.0,21.0,27.0,8.0,0,11.0,3.0,23.0
LO-2,20.0,20.0,23.0,21.0,20.0,21.0,20.0,9.0,11.0,0,0.0,24.0
